In [1]:
import torch
import torchvision as tv

In [2]:
torch.__version__

'1.12.0.dev20220518'

In [3]:
t = torch.tensor([1, 2, 3, 4], device='mps')

In [4]:
t

tensor([1, 2, 3, 4], device='mps:0')

# PETS

In [5]:
import re, math
from types import SimpleNamespace
from pathlib import Path
from tqdm.notebook import tqdm

import wandb
from PIL import Image

import torch
import torch.nn as nn
import torchvision as tv
import torchvision.transforms as T

In [6]:
def get_pets():
    api = wandb.Api()
    at = api.artifact('capecape/pytorch-M1Pro/PETS:v1', type='dataset')
    dataset_path = at.download()
    return dataset_path

In [7]:
dataset_path = get_pets()

wandb: Downloading large artifact PETS:v1, 783.01MB. 25864 files... Done. 0:0:0


In [8]:
class Pets(torch.utils.data.Dataset):
    pat = r'(^[a-zA-Z]+_*[a-zA-Z]+)'
    vocab = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair', 'Egyptian_Mau', 'Maine_Coon', 
             'Persian', 'Ragdoll', 'Russian_Blue', 'Siamese', 'Sphynx', 'american_bulldog', 'american_pit', 
             'basset_hound', 'beagle', 'boxer', 'chihuahua', 'english_cocker', 'english_setter', 'german_shorthaired', 
             'great_pyrenees', 'havanese', 'japanese_chin', 'keeshond', 'leonberger', 'miniature_pinscher', 'newfoundland', 
             'pomeranian', 'pug', 'saint_bernard', 'samoyed', 'scottish_terrier', 'shiba_inu', 'staffordshire_bull', 
             'wheaten_terrier', 'yorkshire_terrier']
    vocab_map = {v:i for i,v in enumerate(vocab)}

    def __init__(self, pets_path, image_size=224):
        self.path = Path(pets_path)
        self.files = list(self.path.glob("images/*.jpg"))
        self.tfms = tfms=T.Compose([T.ToTensor(), T.Resize((image_size, image_size))])
        self.vocab_map = {v:i for i, v in enumerate(self.vocab)}
    
    @staticmethod
    def load_image(fn, mode="RGB"):
        "Open and load a `PIL.Image` and convert to `mode`"
        im = Image.open(fn)
        im.load()
        im = im._new(im.im)
        return im.convert(mode) if mode else im
    
    def __getitem__(self, idx):
        file = self.files[idx]
        return self.tfms(self.load_image(str(file))), self.vocab_map[re.match(self.pat, file.name)[0]]
        
    def __len__(self): return len(self.files)

In [9]:
ds = Pets(dataset_path)

In [10]:
ds[0]

(tensor([[[0.0392, 0.0594, 0.0539,  ..., 0.0322, 0.0322, 0.0322],
          [0.0499, 0.0422, 0.0562,  ..., 0.0158, 0.0230, 0.0172],
          [0.0706, 0.0462, 0.0455,  ..., 0.0456, 0.0426, 0.0353],
          ...,
          [0.1030, 0.1015, 0.1057,  ..., 0.0673, 0.0662, 0.0857],
          [0.0812, 0.0945, 0.1051,  ..., 0.0805, 0.0678, 0.0642],
          [0.1058, 0.0973, 0.1026,  ..., 0.0760, 0.0797, 0.0936]],
 
         [[0.2863, 0.2863, 0.2863,  ..., 0.2701, 0.2701, 0.2701],
          [0.2863, 0.2863, 0.2863,  ..., 0.2667, 0.2670, 0.2680],
          [0.2863, 0.2798, 0.2822,  ..., 0.2689, 0.2694, 0.2706],
          ...,
          [0.2795, 0.2844, 0.2948,  ..., 0.2634, 0.2617, 0.2795],
          [0.2706, 0.2820, 0.2892,  ..., 0.2681, 0.2639, 0.2603],
          [0.2845, 0.2757, 0.2799,  ..., 0.2706, 0.2701, 0.2688]],
 
         [[0.6510, 0.6459, 0.6473,  ..., 0.6436, 0.6436, 0.6436],
          [0.6483, 0.6502, 0.6467,  ..., 0.6428, 0.6399, 0.6457],
          [0.6431, 0.6456, 0.6471,  ...,

In [37]:
def get_dataloader(dataset_path, batch_size, image_size=224):
    "Get a training dataloader"
    ds = Pets(dataset_path, image_size=image_size)
    loader = torch.utils.data.DataLoader(ds, 
                                         batch_size=batch_size,
                                         pin_memory=True,
                                         num_workers=0
                                        )
    return loader

In [38]:
dl = get_dataloader(dataset_path, 32)

In [39]:
for b in tqdm(iter(dl)):
    pass

  0%|          | 0/231 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
def get_model(n_out, arch="resnet18", pretrained=True):
    model = getattr(tv.models, arch)(pretrained=pretrained)
    model.fc = nn.Linear(model.fc.in_features, n_out)
    return model

In [14]:
config = SimpleNamespace(batch_size=32, 
                         dataset_path=dataset_path,
                         device="mps",
                         lr=1e-3,
                         dataset="PETS",
                         epochs=1,
                         model_name="resnet18", 
                         image_size=224,
                        )            

In [15]:
def train(project="pytorch-M1Pro", config=config):
    with wandb.init(project=project, config=config):

        # Copy your config 
        config = wandb.config

        # Get the data
        train_dl = get_dataloader(dataset_path=config.dataset_path, 
                                  batch_size=config.batch_size, 
                                  image_size=config.image_size)
        n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)

        model = get_model(len(train_dl.dataset.vocab), config.model_name)
        model.to(config.device)

        # Make the loss and optimizer
        loss_func = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

       # Training
        example_ct = 0
        step_ct = 0
        for epoch in tqdm(range(config.epochs)):
            model.train()
            for step, (images, labels) in enumerate(tqdm(train_dl, leave=False)):
                images, labels = images.to(config.device), labels.to(config.device)

                outputs = model(images)
                train_loss = loss_func(outputs, labels)
                optimizer.zero_grad()
                train_loss.backward()
                optimizer.step()

                example_ct += len(images)
                metrics = {"train/train_loss": train_loss, 
                           "train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch, 
                           "train/example_ct": example_ct}

                if step + 1 < n_steps_per_epoch:
                    # 🐝 Log train metrics to wandb 
                    wandb.log(metrics)

                step_ct += 1

## On GPU

In [16]:
config.device="mps"
train()

wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_loss,▅█▆▆▄▅▆▃▃▄▅▃▃▃▄▄▃▄▆▄▄▄▃▃▄▄▂▂▄▅▄▁▃▂▂▂▂▂▂▂
train/epoch,0.4329
train/example_ct,3200
train/train_loss,3.61235


KeyboardInterrupt: 

## on CPU

In [ ]:
config.device = "cpu"
train()